In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
import json

Using TensorFlow backend.
/home/rokku/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rokku/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rokku/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rokku/.local/lib/python3.6/site-packages/tensorflow/python/fra

In [2]:
with open('./ALL_SDX_REV.txt', encoding='utf-8') as f:
    text = f.read()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

corpus length: 1231957
total chars: 24
nb sequences: 410639


In [3]:
model = load_model('lermonet_2.h5')

In [4]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [5]:
diversity = 1.0

In [6]:
def generate_rnn(count):
    generated = ""
    start_index = random.randint(0, len(text) - maxlen - 1)
    sentence = text[start_index: start_index + maxlen]
    generated += sentence

    for i in range(count):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
    return generated

In [42]:
print(generate_rnn(2000))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


5A3 A1428B8A4B7Ь8AJ V1264AB4 / S251B9B3A3A S018B V14B6A1A5B6 / A2242B98A S012A4Ь A13A19A2AJ A0126A8 P027Ь V228A37C6Ь C01B C0154AJ S22JB3A / V22649B7A V0349A642A2A4 S1339C64B7B6 / A131A8A2AJ S22C64A8 P011A V011A7 P018B P01A S014C8 P018B3 / S133A8B4A A22A48B / S1262A8A2 P0147A S2219B63C A02JB5B A2224B6A3 S128B9B / S012637C4 C01B7Ь V1319B68JC A041A68B68JB V236A8A64B2 P018B / A239A3A2AJ S016C4Ь V2263A5B P018B S01JA A014A3 / A227C5BJ V226A9B4 A2226B3A A0162AJ V1217A2A7 / V235A69A6A S023A7A64Ь A138A39A63AJ P01B6 A026A5B8 S018B / S238A85B8A P01C A256B8A67A2JB / S023A78C7 S22A8B S011B / S018A6 P012 V226B2C4 P011C C01B V24A37A6B7A / S23649B8B8A A031C63JB V234B96B27B / S22JB3A S238A7Ь1AJC P013 S01A8 C01B / S2242A9A S031A8C5B A138B2A7Ь8A S2217B8C S01A2 C01B / V141B3B4JC A021A4A3 S0119B4 S224A38B P016 S01A8 / A228B1B8 S0168C A0134B V14A4B4A7B6Ь C01B / S1237C1B A148B2B83B8 V221A3A8 A1219B54B V024B4A4Ь S01A8 S026B3B3 S019A6 C01B / S039AJ3B8A V22649A64A7 P011A S018A8 P011A V018A3 C01B C018B V141A4B95

In [7]:
def levenshtein_distance(word1, word2):
    if len(word1) < len(word2):
        return levenshtein_distance(word2, word1)

    if len(word2) == 0:
        return len(word1)

    previous_row = list(range(len(word2) + 1))

    for i, char1 in enumerate(word1):
        current_row = [i + 1]

        for j, char2 in enumerate(word2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (char1 != char2)

            current_row.append(min(insertions, deletions, substitutions))

        previous_row = current_row
    return previous_row[-1]

In [8]:
with open('SDX_WORD.txt', 'r') as f:
    sdx_word = json.loads(f.read())

In [9]:
sdx_word

{'A1339B8C5BJ': 'гремучий',
 'V14A4B2A7B6Ь': 'одевались',
 'S237A81A4AJ': 'лампадой',
 'A034B3JA': 'тихая',
 'A145A81A863A3A': 'шампанского',
 'S141A7A5B8JB': 'положенье',
 'V139A6C4B2': 'рассудив',
 'S131A4A53B': 'подачки',
 'V2261B5A': 'спеша',
 'A048C5B4B7Ь8A': 'мучительна',
 'A233A3A2A': 'каково',
 'A133A9A5B': 'короче',
 'V1319B8B4A': 'приметя',
 'S013A1Ь': 'копь',
 'V134A9A1B5Ь': 'торопишь',
 'V035B15B6A': 'шепчется',
 'A2263A7AJ': 'скалой',
 'S141A4A9A5B3': 'подарочек',
 'V13B61A78B4Ь': 'исполнить',
 'V032A649B7B7': 'выстрелил',
 'A04JA6A5B63AJ': 'языческой',
 'A141A6A98JA': 'позорная',
 'S021A7Ь1A': 'больпо',
 'V2369B19B7A6Ь': 'сребрилась',
 'V241A6A2B4B': 'позовите',
 'V236A2B98C4': 'завернут',
 'V2419B8A6B7A': 'приносила',
 'V2419B8A47B5A4': 'принадлежат',
 'A248B6B81JB': 'неземпые',
 'V2419B42B5JB4': 'предвещает',
 'S22A2A6': 'афос',
 'V136A8B4B8': 'заметим',
 'V142A7A3B4AJ': 'волокитой',
 'A1239B3B': 'греки',
 'V13B61A2B': 'избави',
 'V226JA7': 'сеял',
 'A038B58A8B': 'нежны

In [10]:
from russtress import Accent
from pyphonetics import Soundex
from fonetika.soundex import RussianSoundex
import rusyllab
from fonetika.distance import PhoneticsInnerLanguageDistance
from collections import defaultdict

In [11]:
accent = Accent()
soundex = RussianSoundex(delete_first_letter=True, code_vowels=True)
phon_distance = PhoneticsInnerLanguageDistance(soundex)

In [12]:
def get_rhyme_ending(word):
    stress_pos = accent.put_stress(word).find('\'')
    if stress_pos == -1:
        return word

    lst = list(word)
    lst[stress_pos-1] = lst[stress_pos-1].upper()
    word = ''.join(lst)
    sx = rusyllab.split_words([word])
    for i in range(len(sx)):
        if not sx[i].islower():
            return ''.join(sx[i:]).lower()
def is_rhyme(word1, word2):
    end1 = get_rhyme_ending(word1)
    end2 = get_rhyme_ending(word2)
    if phon_distance.distance(end1, end2) <= 1:
        return True
    else:
        return False
def check_rhyme_end_with_word(end, word):
    word_end = get_rhyme_ending(word)
    if phon_distance.distance(end, word_end) <= 1:
        return True
    else:
        return False

In [47]:
content = sdx_word
style_corpus_soundex = sdx_word
cache = set()
lines = []
rhymed_ends = defaultdict(list)
iteration = 0
while iteration < 1:
    iteration += 1
    lines = generate_rhyme(3000, 'люблю грозу в начале мая').split("/")
    print("generated", len(lines))
    for line in lines:
        terms = [t for t in line.split(" ") if t]
        if not terms:
            continue
        
        generated_line = []
        for term in terms[:-1]:
            guessed_words = {}
            for idx, word in content.items():
                lev_dist = levenshtein_distance(idx, term)
                if len(word) > 1 and lev_dist <= 1:
                    guessed_words[word] = lev_dist
            
            if not guessed_words:
                for idx, word in style_corpus_soundex.items():
                    lev_dist = levenshtein_distance(idx, term)
                    if lev_dist <= 1:
                        guessed_words[word] = lev_dist
                        
            if guessed_words:
                generated_line.append(sorted(guessed_words, key=guessed_words.get)[0])
        
        last_words = []
        last_term = terms[-1]
        for idx, word in content.items():
            lev_dist = levenshtein_distance(idx, last_term)
            if len(word) > 1 and lev_dist <= 1:
                last_words.append(word)
                
        generated_line_str = " ".join(generated_line)
        for last_word in last_words:
            if last_word in rhymed_ends:
                continue
            
            rhymed_end = get_rhyme_ending(last_word)
            for word in rhymed_ends.keys():
                if check_rhyme_end_with_word(rhymed_end, word):
                    rhymed_ends[word].append("{} {}".format(generated_line_str, last_word))
                    break
            else:
                rhymed_ends[last_word].append("{} {}".format(generated_line_str, last_word))
            
    print("Iteration {} completed".format(iteration))
        

S128JA S138A5A7B P012 S2239A6C V227C17C


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


generated 74
Iteration 1 completed


In [48]:
for values in rhymed_ends.values():
    if len(values) > 3:
        print("-" * 30)
        for value in values:
            print(value)

------------------------------
мея начале во грозу люблю
мея начале во грозу куплю
страстей просится мне страдай ну
страстей просится мне страдай ли
земли долго сердце во как дотла
своей ним из забытом мне ла
окно мне потеря толпу
земли забытом ну
земли забытом ли
стрела мне пали вещий ну
стрела мне пали вещий ли
звук долго безвестный дух долго лн
как такой ним из лн
свою ним из земля
ним из мя спешу
толпы кровь прекрасных земли
толпы кровь прекрасных земля
главе бедное стане ла
главе бедное стане ну
земли русские слеза ну
земли русские слеза ли
страстей ним из бросили ну
страстей ним из бросили ли
собрать полными мне смею ну
собрать полными мне смею ли
страстей того смел ну
страстей того смел ли
------------------------------
страстей просится мне страдай меж
страстей просится мне страдай мне
страстей просится мне страдай ни
страстей просится мне страдай нет
ты забыты зверь унижу
земли забытом мне
земли забытом ни
земли забытом нет
проходит сердце как ценил
земли как забытом ни мех
зе

In [59]:
rhymed_ends

defaultdict(list,
            {'мея': ['люблю грозу во начало мея',
              'люблю грозу во начало нею',
              'люблю грозу во начало боя',
              'люблю грозу во начало муя',
              'люблю грозу во начало чуя',
              'и долго мя оставит мя',
              'со себя пали мя'],
             'тебе': ['стоял он тебе',
              'стоял он пебо',
              'стоял он себя',
              'стоял он тебя',
              'садится мех земля ни знающей земле',
              'садится мех тебе столбы',
              'мя бросали такой запел',
              'и голос странник как земле',
              'со меня садится труд прости',
              'со меня садится труд проси',
              'со меня садится труд пройти',
              'его браниться мир давно запел',
              'как голос скачут голос мне',
              'как голос скачут голос бег',
              'ни мя стоял со мне',
              'со себя пали мне',
              'со себя пали мпе',
     

In [61]:
s = 'люблю грозу в начале мая'
for item in rhymed_ends:
    if is_rhyme(item, 'мая'):
        for line in rhymed_ends[item]:
            print(s)
            print(line)
            print('-'*30)

люблю грозу в начале мая
люблю грозу во начало мея
------------------------------
люблю грозу в начале мая
люблю грозу во начало нею
------------------------------
люблю грозу в начале мая
люблю грозу во начало боя
------------------------------
люблю грозу в начале мая
люблю грозу во начало муя
------------------------------
люблю грозу в начале мая
люблю грозу во начало чуя
------------------------------
люблю грозу в начале мая
и долго мя оставит мя
------------------------------
люблю грозу в начале мая
со себя пали мя
------------------------------


In [13]:
import nltk
def encode(word):
    sdx = soundex.transform(word)

    accent_letter = accent.put_stress(word).find('/')
    rel_accent = accent_letter / len(word)
    num_syl = len(rusyllab.split_words(word.split()))
    if rel_accent <= 0.4:
        rel_accent = 0
    elif rel_accent <= 0.67:
        rel_accent = 1
    else:
        rel_accent = 2

    _, pos = nltk.pos_tag([word], lang="rus")[0]
       
    if pos in ("NONLEX"):
        return "-{}{}{}".format(rel_accent, num_syl, sdx)
    return "{}{}{}{}".format(pos[0], rel_accent, num_syl, sdx)

In [30]:
def encode_sent(sent, size):
    encoded = []
    for word in sent.split():
        encoded.append(encode(word))
    encoded_str =  ' '.join(encoded[::-1])
    print(encoded_str)
    encoded_str += ' /'
    if len(encoded_str) >= size:
        return encoded_str[len(encoded_str)-size:]
    else:
        return (' ' * (size-len(encoded_str))) + encoded_str

In [31]:
encode_sent('люблю грозу в начале мая', 40)

S128JA S138A5A7B P012 S2239A6C V227C17C


'128JA S138A5A7B P012 S2239A6C V227C17C /'

In [33]:
encode('во')

'P012A'

In [37]:
words_list = list(sdx_word.values())

In [39]:
len(words_list)

22202

In [41]:
rhymed = []
for i, word in enumerate(words_list):
    if i % 100 == 0:
        print(i)
    if is_rhyme(word, 'мая'):
        rhymed.append(word)

0
100
200
300
400
500
600


KeyboardInterrupt: 

In [ ]:
print(rhymed)

In [44]:
def generate_rhyme(count, sent):
    generated = ""
    sentence = encode_sent(sent, maxlen)
    generated += sentence

    for i in range(count):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, 0.2)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
    return generated

In [45]:
s = 'люблю грозу в начале мая'

In [50]:
generate_rhyme(1000, s)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


'227C17C S2239A6C P012 S138A5A7B S128JA \\ S01JA V011A7 S226B87B A1319B48B58A V2319A6B7B \\ C01B S022B49A8 S025B68B A141B6A1A4A8 \\ A013A3 S01JA V136A4B4B4 P018B V22649JBJ \\ C01B A024B3BJ S0164A7 A013A3 S023A7A6 A021A58AJ S0168A3 A1319B64C18AJ S01649A3 \\ P018B V23A439JC4 S01A8 V2319A3A5B4 V22649A4A4Ь \\ C01B S023A7A6 S014B3 A131A3A58AJ S0167A2 \\ V2239A6A2 C01B S023A7A6 S12649A64B S0162B4 \\ A224A3AJ S226B87B A1319B4A48A V1219B4AJ S0126A9 \\ C01B S021B42A3 A021B7Ь8AJ S018B3 S2264A9A3 \\ A013A3 S023A7A6 S12649A8B3 V2319A1B5A7 \\ S01JA V2319A2B4B7 P016A S018B3 S138A75A8JB \\ S2239A6AJ S01A8 V021A8B6 P012 S018BJ A2262JC \\ A013A3 S023A7A6 V12639A5B4 S013A9 \\ S2237A6A V133A6A7A6Ь S0134A P016A S018B3 \\ A013A3 S01JA V13C68JB4 S025B642A3 S0168A3 \\ A22A4B8 P018B V1264A8B4 S01JA S22A8A \\ S226B87A A2368A3A8A S227B6A V223A9B4 \\ S01A8 V2319A2B7B A234B5B8A S014A \\ S22A8A V136A4B6A S018B3 S2264A9B3 \\ S01JA V0168A7 A138B2B48A V14C68B8A7A \\ C01B V022B4B7 S01A8 A133A4A9AJ S018B3 \\ A013A3 S01

In [46]:
content = sdx_word
style_corpus_soundex = sdx_word
generated = generate_rhyme(1000, s).split()
print(generated)
for term in generated:
    if term == "/":
        print(term)
        continue
        
    guessed_words = {}
    for idx, word in content.items():
        lev_dist = levenshtein_distance(idx, term)
        if len(word) > 1 and lev_dist <= 1:
            guessed_words[word] = lev_dist
           
    if not guessed_words:
        for idx, word in style_corpus_soundex.items():
            lev_dist = levenshtein_distance(idx, term)
            if lev_dist <= 1:
                guessed_words[word] = lev_dist
    
    print(sorted(guessed_words, key=guessed_words.get)[0] if guessed_words else "", end=" ")

S128JA S138A5A7B P012 S2239A6C V227C17C


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


['128JA', 'S138A5A7B', 'P012', 'S2239A6C', 'V227C17C', '/', 'V22649A4A4Ь', 'A038B58JB', 'A028A5B8', 'P018B', '/', 'S226B87B', 'A031B68JC', 'P016', 'V131A4B5BJ', '/', 'S22649A64BJ', 'V1319B1B5B4', 'S018B8', 'P016', 'A228A58A', '/', 'S22649A64BJ', 'V131A3A8B5Ь', 'S014A', 'S014A', '/', 'S014A', 'V131A39A7A6Ь', 'A013A3', 'V121A7B', 'P012', 'C01B', '/', 'S226A1A4', 'A014A4', 'S138A8B8JB', 'S018B', 'P013A', '/', 'S226B87B', 'A031B9B3AJ', 'A028A5B8', 'P018B', '/', 'S0162C3', 'A024A73A', 'A024A73A', 'S138A4B7B', 'P012', '/', 'S226B1B', 'P012', 'V22649A6A', 'V22649A64AJ', 'S018B8', 'P016', 'C0154A', '/', 'S018B8', 'P016', 'V131A4B67B6Ь', 'S018B', 'V131A4B9A', '/', 'V226B4B4', 'S018A6', 'P018A', 'V226B4B4', 'S018B8', 'P016', 'C01B', '/', 'S0162C3', 'A023A9Ь3B3', 'S1319A64B8JB', 'S018B', 'V231A4A8C7', '/', 'S0162A8', 'A028A5B8', 'S018B', 'V2219B64A7', 'S018B8', 'P016', 'C01B', '/', 'S014C3', 'A024A73A', 'V131A4C8A', 'A224A3A', '/', 'S0126A9', 'V13C2B9B4', 'S018B8', 'P016', 'V131A4A75B8', 'A013A3'